# Comparison Between TreeValue and DM-Tree

In this section, we will compare the feature and performance of the [dm-tree](https://github.com/deepmind/tree) library, which is developed by deepmind.

Before starting the comparison, let us define some thing.

In [ ]:
origin = {'a': 1, 'b': 2, 'c': {'x': 3, 'y': 4}}

## Mapping Operation

Mapping operation is quite common in the processing of trees. A mapping function should be provided to create a new tree based on the mapped tree.

### TreeValue's mapping

In TreeValue, mapping is provided to simply create a new tree.

In [ ]:
from treevalue import FastTreeValue, mapping

tv = FastTreeValue(origin)
tv

In [ ]:
mapping(tv, lambda x: x * 2)

Here is the performance test.

In [ ]:
%timeit mapping(tv, lambda x: x * 2)

In order to support the cased that the mapped value is related to both path and value of each node, we can use the 'path mapping mode' by simply use the second parameter of the mapping function.

In [ ]:
mapping(tv, lambda x, p: ('path', p, 'value', x))

And here is the performance

In [ ]:
%timeit mapping(tv, lambda x, p: ('path', p, 'value', x))

### DM-Tree's mapping

In DM-Tree, mapping operation is supported by [map_structure](https://tree.readthedocs.io/en/latest/api.html#tree.map_structure) function.

In [ ]:
from tree import map_structure

In [ ]:
map_structure(lambda x: x * 2, origin)

This is the performance of `map_structure`, obviously much slower than `mapping` in TreeValue.

In [ ]:
%timeit map_structure(lambda x: x * 2, origin)

To supported the second situation in the last section, [map_structure_with_path](https://tree.readthedocs.io/en/latest/api.html#tree.map_structure_with_path) can be used.

In [ ]:
from tree import map_structure_with_path

In [ ]:
map_structure_with_path(lambda path, x: ('path', path, 'value', x), origin)

Here is the performance.

In [ ]:
%timeit map_structure_with_path(lambda path, x: ('path', path, 'value', x), origin)

## Flatten and Unflatten

In tree operations, flatten is often used to linearly expand the tree structure for operations such as parallel processing. Based on flatten, unflatten is its inverse operation, which can recover the tree structure from the linear data.

### TreeValue's Performance

In TreeValue, flatten and unflatten are provided, which usage are simple.

In [ ]:
from treevalue import FastTreeValue, flatten, unflatten

origin_tree = FastTreeValue(origin)
origin_tree

In [ ]:
flatted = flatten(origin_tree)
flatted

Here is the performance of `flatten`

In [ ]:
%timeit flatten(origin_tree)

The tree can be re-created from `flatted` with function `unflatten`.

In [ ]:
unflatten(flatted)

And here is the performance.

In [ ]:
%timeit unflatten(flatted)

### DM-Tree's Performance

[Flatten](https://tree.readthedocs.io/en/latest/api.html#tree.flatten) is provided in DM-Tree as well, but it differs from that in TreeValue, as the following

In [ ]:
from tree import flatten

flatten(origin)

Here is the performance

In [ ]:
%timeit flatten(origin)

The structure of the tree is dropped, only the data is extracted from the tree. This means the `flatten` function in DM-Tree is irreversible, we can not recover the original tree with the result above.

The reversible resolution provided in DM-Tree is [flatten_with_path](https://tree.readthedocs.io/en/latest/api.html#tree.flatten_with_path)

In [ ]:
from tree import flatten_with_path

flatten_with_path(origin)

Here is the performance, much slower than `flatten` in TreeValue.

In [ ]:
%timeit flatten_with_path(origin)

To re-create the original tree, we need a tree structure with any objects filled inside. Use the [unflatten_as](https://tree.readthedocs.io/en/latest/api.html#tree.unflatten_as) to archive this goal.

In [ ]:
from tree import unflatten_as

unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

Here is the performance.

In [ ]:
%timeit unflatten_as({'a': None, 'b': None, 'c': {'x': None, 'y': None}}, [1, 2, 3, 4])

### Positional Replacement

It is obvious that the `unflatten_as` in DM-Tree is quite different from `unflatten` in TreeValue, for the former one is `replacing` and the latter one is `constructing`. This means the `unflatten_as` in DM-Tree may supported some more features, such as creating a new tree with another tree's structure and the given values. So we need an experiment on this, called 'positional replacement'.

First, in TreeValue, we can build a function named `replace` to realize this.

In [ ]:
from treevalue import flatten, unflatten

def replace(t, v):
    pairs = flatten(t)
    return unflatten([(path, vi) for (path, _), vi in zip(pairs, v)])

Create a new tree based on `origin_tree`'s structure and new values.

In [ ]:
replace(origin_tree, [3, 5, 7, 9])

Here is the performance.

In [ ]:
%timeit replace(origin_tree, [3, 5, 7, 9])

In DM-Tree, `unflatten_as` can be directly used.

In [ ]:
from tree import unflatten_as

unflatten_as(origin, [3, 5, 7, 9])

Here is the performance, even much slower than the `replace` function for integration.

In [ ]:
%timeit unflatten_as(origin, [3, 5, 7, 9])

## Conclusion

The mapping operation is supported by both library, and `mapping` in TreeValue's performance is significantly higher than the `map_structure` and `map_structure_with_path` in DM-Tree.

The `flatten` and `unflatten` in TreeValue is reversible, but in DM-Tree not. DM-Tree's performance on flatten and unflatten operation is lower than that in TreeValue in all aspects.